In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [4]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(35856, 674)
(35856,)
(20252, 674)
(20252,)


In [5]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,12,0,0,0,0,0,4,0,0,0,...,0,0,6,0,0,0,0,0,2024-01-29 00:00:00,HB101
1,10,0,0,0,0,0,6,0,0,0,...,0,0,11,0,0,0,0,0,2024-01-29 12:00:00,HB101
2,0,0,0,0,0,0,16,0,0,0,...,0,0,15,0,0,0,0,0,2024-01-30 00:00:00,HB101
3,29,0,0,0,0,0,11,0,0,0,...,0,0,9,0,0,0,0,0,2024-01-30 12:00:00,HB101
4,0,0,0,0,0,0,19,0,0,0,...,0,0,13,0,0,0,0,0,2024-01-31 00:00:00,HB101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35851,30,0,0,0,0,0,39,0,0,0,...,0,0,13,0,0,0,0,0,2024-08-29 12:00:00,JC116
35852,1,0,0,0,0,0,7,0,0,0,...,0,0,25,0,0,0,0,0,2024-08-30 00:00:00,JC116
35853,42,0,0,0,0,0,18,0,0,0,...,0,0,5,0,0,0,0,0,2024-08-30 12:00:00,JC116
35854,9,0,0,0,0,0,14,0,0,0,...,0,0,25,0,0,0,0,0,2024-08-31 00:00:00,JC116


In [6]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

In [7]:
pipeline

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x14f6ad940>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [8]:
# Get your pipeline
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

# Train
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000941 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10989
[LightGBM] [Info] Number of data points in the train set: 35856, number of used features: 115
[LightGBM] [Info] Start training from score 7.468178


In [9]:
y_pred

array([11.24552291, 23.55042733,  2.784709  , ..., 14.39806549,
        0.87934847, 19.5175935 ])

In [10]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, y_pred)
print(f"{test_mae:.4f}")

2.5742


In [11]:
import joblib
import src.config as config

# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['/Users/ajit/Desktop/Citibike_prediction_aml/models/lgb_model.pkl']

In [12]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

2025-05-10 16:38:59,562 INFO: Initializing external client
2025-05-10 16:38:59,562 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 16:39:00,636 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231000


In [13]:
# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

Error creating feature view: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1231000/featurestores/1214534/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270179,"usrMsg":"Feature view: time_series_hourly_feature_view_citi_bike, version: 1","errorMsg":"The provided feature view name and version already exists"}', error code: 270179, error msg: The provided feature view name and version already exists, user msg: Feature view: time_series_hourly_feature_view_citi_bike, version: 1
Feature view 'time_series_hourly_feature_view_citi_bike' (version 1) retrieved successfully.


In [14]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name=config.MODEL_NAME,
    metrics={"test_mae": test_mae},
    description="LightGBM model for bike rides",
    input_example=X_train.sample(),
    model_schema=model_schema,
   
)
# https://community.hopsworks.ai/t/attributeerror-windowspath-object-has-no-attribute-startswith/1003
model.save(str(config.MODELS_DIR) + "/lgb_model.pkl")
# model.save('C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/ajit/Desktop/Citibike_prediction_aml/models/lgb_model.pkl: 0.000%|          | 0/309419 elapse…

Uploading /Users/ajit/Desktop/Citibike_prediction_aml/notebooks/input_example.json: 0.000%|          | 0/2097 …

Uploading /Users/ajit/Desktop/Citibike_prediction_aml/notebooks/model_schema.json: 0.000%|          | 0/48706 …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1231000/models/citibike_hourly_predictor_model/1


Model(name: 'citibike_hourly_predictor_model', version: 1)